# Introduction

This notebook explores how we might use GPT-3 to parse irregular tables in Excel spreadsheets. It explores singe- and few-shot learning 
approaches, as well as model fine-tuning.

# Setup

1. See variables 'filename' and 'prompt_sample_table' below for the sample file used in exploring prompts, you can change this and try your own.

2. The data used for fine-tuning comes from the [Humanitarian Data Exchange (HDX)](https://data.humdata.org/) project, specifically files related
to Kenya. You can find details for how to download these files in this [blog post](https://towardsdatascience.com/predicting-metadata-for-humanitarian-datasets-using-gpt-3-b104be17716d), the notebook for which is in this repo. This produces a pkl file of spreadsheet resources, the location
of which below is set with variable 'hdx_resources_pkl_file'

3. For running OpenAI you will need to create a file called `api_key.txt` in the current directory and put your [OpenAI API key](https://beta.openai.com/account/api-keys) in there (just the API string, nothing else)

A conda environment is provided for running these notebooks, please see `environment.yml` for more details.

In [ ]:
%pip install sklearn
%pip install xlsx2html
%pip install openpyxl
%pip install bs4


In [27]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import json
import os
import chardet
import shutil
import sys
import re
import traceback
import zipfile
import time

import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from IPython.display import display, Markdown, Latex

import openai as ai
from openai import cli

import openpyxl
from openpyxl.utils import range_boundaries

from io import StringIO

# Open AI API key should be put into this file
ai.api_key_path = "./api_key.txt"

# File used for prompt examples, as created by [this notebook]((https://github.com/datakind/gpt-3-meta-data-discovery/blob/main/hdx_gpt-3_tag_prediction.ipynb)
filename = "./data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx"

# A similar but slightly different sheet to filename, used as example in prompt
prompt_sample_table1 = './data/Kenya/kenya-infant-dietary-statistics-in-bomet-county_0f08f14e-07ca-4fff-84c0-2568ea6c793a/infants_on_Minimum_Dietary_Diversity_xlsx_a7838f28_9f77_48eb_99b5_22f3dad9b893/infants on Minimum Dietary Diversity.xlsx'
#prompt_sample_table1 = './data/infants on Minimum Dietary Diversity.xlsx'

# A PKL file containing a list of spreadsheet resources from HDX, as created by [this notebook]((https://github.com/datakind/gpt-3-meta-data-discovery/blob/main/hdx_gpt-3_tag_prediction.ipynb)
hdx_resources_pkl_file = "./data/Kenya_data_details.pkl"

# Number of rows in table header to use when analyzing headers
maxn = 15

# For splitting HDX data into train (0:max_dataset_orgs) and test (max_dataset_orgs:) 
max_dataset_orgs = 10


In [2]:
def get_sheet_attributes(sheet, maxn):
    """
    Returns a set of table attributes for a given sheet

    Input Parameters:
        sheet: Obj
            Openpyxl sheet object
        maxn: int
            Number of rows to scan at start of sheet

    Returns:
        null_cells_in_rows: list of ints
            Count of NULL records in forst maxn rows
        float_cells_in_rows: list of ints
            Count of numeric records in first maxn rows
        unique_vals_in_rows: list of ints
            Count of unique values in first maxn rows
        year_vals_in_rows: list of ints
            Count of year values in first maxn rows
        hxl_row: int
            Row number of HXL header row
        first_float_row: int
            Row number of row with most numeric records
        first_not_null_row: int
            Row number of row with most non-null records

    """
    dd = pd.DataFrame(sheet.values)

    null_cells_in_rows = list(
        dd[0:maxn].apply(lambda x: x.isnull().sum(), axis="columns")
    )
    float_cells_in_rows = []
    unique_vals_in_rows = []
    year_vals_in_rows = []
    report_json = {}
    hxl_row = None
    for index, row in dd[0:maxn].iterrows():
        unique_vals = list(row.unique())
        unique_vals = [
            i for i in unique_vals if i is not None and str(i) != "nan"]
        unique_vals_in_rows.append(len(unique_vals))
        float_count = 0
        year_count = 0
        if check_hdx_header(list(row)):
            hxl_row = index
        for col in dd.columns:
            val = row[col]
            # Handle numbers that come through as strings
            if isinstance(val, str):
                val = val.replace(",", "").replace(" ", "")
                if val.isnumeric():
                    val = int(val)
            # Check for year values
            if (
                ((isinstance(val, int) or isinstance(val, float)) and val % 1 == 0)
                and val > 1900
                and val < 2100
            ):
                year_count += 1
                continue
            # Check for HXL tags
            if isinstance(val, float) or isinstance(val, int) or "^=" in str(row[col]):
                float_count += 1
        float_cells_in_rows.append(float_count)
        year_vals_in_rows.append(year_count)

    max_floats = max(float_cells_in_rows)
    min_nulls = min(null_cells_in_rows)
    first_float_row = 0
    if sum(float_cells_in_rows) > 0:
        for i in range(1, len(float_cells_in_rows)):
            # Use a ratio or special case where we go from zero to some
            if float_cells_in_rows[i] / max_floats > 0.5 or (
                float_cells_in_rows[i] > 0 and float_cells_in_rows[i - 1] == 0
            ):
                first_float_row = i
                break
    first_not_null_row = np.argmin(null_cells_in_rows)

    report = f"Nulls in first {maxn} rows: {str(null_cells_in_rows)}\n"
    report += f"Numeric first {maxn} rows: {str(float_cells_in_rows)}\n"
    report += f"Unique values in first {maxn} rows: {str(unique_vals_in_rows)}\n"
    report += f"Year values in first {maxn} rows: {str(year_vals_in_rows)}\n"
    report += f"HXL row: {str(hxl_row)}\n"

    report += f"\nFirst reduced nulls row: {str(first_not_null_row)}\n"
    report += f"First increased numeric row (excluding years): {str(first_float_row)}\n"

    report_json = {
        "null_cells_in_rows": null_cells_in_rows,
        "float_cells_in_rows": float_cells_in_rows,
        "unique_vals_in_rows": unique_vals_in_rows,
        "year_vals_in_rows": year_vals_in_rows,
        "hxl_row": hxl_row,
        "first_float_row": first_float_row,
        "first_not_null_row": first_not_null_row,
    }

    return report, report_json


def check_hdx_header(first_row):
    """
    This function checks if the first row of a csv file likely an HDX header.
    """
    matches = ["#meta", "#country", "#data", "#loc", "#geo"]
    first_row = str(first_row)
    if any(x in first_row for x in matches):
        return True
    else:
        return False


def pad_merged_cells(sheet):
    """
    Unmerge merged cells and fill with merged value.

    Input Parameters
    ----------------
    sheet: Obj
        Openpyxl sheet object

    Output Parameters
    -----------------
    df: Dataframe
        Pandas dataframe of the table
    """

    dd = pd.DataFrame(sheet.values)

    # Scan for maxn rows
    maxn = 10

    hasmerged = False
    if len(sheet.merged_cells.ranges) > 0:
        hasmerged = True

    if hasmerged:
        merge_list = []
        for merge in sheet.merged_cells.ranges:
            merge_list.append(merge)

        for cell_group in merge_list:
            min_col, min_row, max_col, max_row = range_boundaries(
                str(cell_group))
            top_left_cell_value = sheet.cell(row=min_row, column=min_col).value
            sheet.unmerge_cells(str(cell_group))
            for row in sheet.iter_rows(
                min_col=min_col, min_row=min_row, max_col=max_col, max_row=max_row
            ):
                for cell in row:
                    cell.value = top_left_cell_value

    # Extract data and save to dataframe
    data = []
    for row in sheet.iter_rows(min_row=1):
        row_data = []
        for cell in row:
            if cell.value is None:
                row_data.append(None)
            else:
                row_data.append(cell.value)
        if any(row_data):
            data.append(row_data)

    df = pd.DataFrame(data)

    # Remove duplicate columns
    df = df.T.drop_duplicates().T

    # Remove duplicate rows
    df = df.drop_duplicates()

    # Fill NaN with blank string for easier viewing
    df = df.fillna("")

    return df, sheet, hasmerged


def find_first_data_row(sheet, maxn, model=None):
    """
    Find the first row with data in it

    Input Parameters:
        sheet: Obj
            Openpyxl sheet object
        maxn: int
            Number of rows to scan at start of sheet
        model: str
            GPT-3 model to use to find first datarow. If not set, will use simple rules

    Returns:
        datarow: int
            Row number of first data row
        report: str
            Report on table attributes

    """

    report, report_json = get_sheet_attributes(sheet, maxn)

    if model == None:
        # Make a guess at which row is the data row
        datarow = report_json["first_not_null_row"]
        # Sometimes we have header rows where none are null, in this case we want to use the row with the most floats
        if report_json["first_float_row"] > datarow:
            datarow = report_json["first_float_row"]
        # HXL row is always the row before the data row
        if report_json["hxl_row"] is not None:
            datarow = report_json["hxl_row"]
        # If we a row with a lot of year values below datarow, use that
        if report_json["year_vals_in_rows"][datarow] > 3:
            datarow = datarow + 1
    else:
        print("Call GPT-3 to find first data row")
        report_elements = report.split("\n\n")
        prompt = (
            report_elements[0]
            + "\n\n"
            + report_elements[1]
            + "\n\nData starts at row: "
        )
        print(prompt)
        res = make_gpt3_prediction(prompt, model, temperature=0.0)
        datarow = int(res["predicted"].strip())
        print("GPT-3 prediction: " + str(datarow))

    return datarow, report


def make_gpt3_prediction(prompt, model, temperature=0.99, max_tokens=13):
    """
    Wrapper to call GPT-3 to make a prediction (completion) on a single prompt.
    Also calls post_process() to clean up the prediction.

    Parameters
    ----------
    prompt : str
        Prompt to use for prediction
    model : str
        GPT-3 model to use
    temperature : float
        Temperature to use for sampling
    max_tokens : int
        Maximum number of tokens to use for sampling

    Returns
    -------
    result : dict
        Dictionary with prompt, predicted, predicted_post_processed
    """
    result = {}
    result["prompt"] = prompt
    model_result = ai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n"],
        logprobs=1,
    )
    result["predicted"] = model_result["choices"][0]["text"].replace(" ", "")
    result["logprobs"] = model_result["choices"][0]["logprobs"]["top_logprobs"]
    return result


def parse_excel_sheet(sheet, maxn=10, model=None, debug=True):
    """
    Excel reader which will attempt to identify table on sheet and return a dataframe
    with the correct columns, with merged column headers collapsed into one row.

    Collapsed headers are a concatenation of all header strings up to the data row. So
    for example, if the header is:

    A | B | C

    The collapsed header is:

    'A - B - C'

    Note: Multiple tables on a sheet are not currently supported, but are detected and
    header rows include a prefix to beware of this data.

    Input Parameters
    ----------------
    sheet: Obj
        Openpyxl sheet object
    maxn: int
        Number of rows to scan at start of sheet
    model: str
        GPT-3 model to use for finding first data row, if not set use simple rules
    debug: bool
        Print debug information

    Output Parameters
    -----------------
    df: Dataframe
        Pandas dataframe of the table
    hasmerged: bool
        Indicates whether table has merged cells
    cells_report: str
        Report of row/column attributes in table
    """

    if debug:
        before = pd.DataFrame(sheet.values)[0:maxn]

    # Pad merged cells
    df, sheet, hasmerged = pad_merged_cells(sheet)

    # Identify row where data starts using GPT-3
    if model != None:
        method = 1
    datarow, report = find_first_data_row(sheet, maxn, model=model)

    # Find first row of varying data (we will skip header rows where all the same)
    min_row = 2
    for row in sheet.iter_rows(min_row=1, max_row=3):
        left_value = sheet.cell(row=row[0].row, column=1).value
        all_same = True
        for cell in row:
            if cell.value != left_value and cell.value != None:
                all_same = False
                break
        if all_same:
            min_row = min_row + 1

    # Collapse merged headings (assumes we used 'fill')
    if datarow > 0 and hasmerged:
        datarow = datarow + 1  # Convert to excel row number
        for row in sheet.iter_rows(min_row=min_row, max_row=datarow - 1):
            for cell in row:
                val_m1 = str(cell.offset(-1, 0).value)
                val_here = str(cell.value)
                if val_here not in val_m1:
                    cell.value = val_m1 + " - " + str(cell.value)
                else:
                    cell.value = val_m1
                cell.value = re.sub("^ +-|None", "", cell.value)

        # Extract data and save to dataframe
        data = []
        for row in sheet.iter_rows(min_row=datarow - 1):
            row_data = []
            for cell in row:
                if cell.value is None:
                    row_data.append(None)
                else:
                    row_data.append(cell.value)
            if any(row_data):
                data.append(row_data)

        df = pd.DataFrame(data)
    else:
        df = pd.DataFrame(sheet.values)

    # Remove duplicate columns
    df = df.T.drop_duplicates().T

    # Pad blank cells
    df = df.fillna("")

    # Make first row column headings
    df.columns = df.iloc[0]
    df = df[1:]

    if debug:
        display(before)
        print(f"\n{report}")
        display(df.iloc[0:3])

    return df, hasmerged, report


def output_prediction_metrics(results, prediction_field="predicted_post_processed"):
    """
    Prints out model performance report if provided results in the format:

    [
        {
            'prompt': ' \'ISO3\' | "[\'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\', \'RWA\']"',
            'predicted': ' #country+code+iso3+v_iso3+',
            'expected': '#country+code'
        },
        ... etc ...
    ]

    Parameters
    ----------
    results : list
        See above for format
    prediction_field : str
        Field name of element with prediction. Handy for comparing raw and post-processed predictions.
    """
    y_test = []
    y_pred = []
    for r in results:
        if "expected" not in r:
            print("Provided results do not contain expected values.")
            sys.exit()
        y_pred.append(r[prediction_field])
        y_test.append(r["expected"])

    print(f"There were {len(y_test)} predictions made.")
    print(f"\nPrediction using field {prediction_field} ...\n")
    print(f"Accuracy: {round(accuracy_score(y_test, y_pred),2)}")
    print(
        f"Precision: {round(precision_score(y_test, y_pred, average='weighted', zero_division=0),2)}"
    )
    print(
        f"Recall: {round(recall_score(y_test, y_pred, average='weighted', zero_division=0),2)}"
    )
    print(
        f"F1: {round(f1_score(y_test, y_pred, average='weighted', zero_division=0),2)}"
    )

    return


In [3]:
df = pd.read_excel(filename, sheet_name="Sheet1")
df = df.fillna("")
display(df)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,,Sub county,,,,,,,
2,,,,,Chepalungu,,,,Bomet Central,,,
3,,,,,Male,,Female,,Male,,Female,
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that ...,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%


In [5]:
df.to_csv("test.csv")
print(df.to_csv())


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,,Sub county,,,,,,,
2,,,,,Chepalungu,,,,Bomet Central,,,
3,,,,,Male,,Female,,Male,,Female,
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
10,,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%



In [6]:
with open("test.csv", "r") as file:
    csv_as_str = file.read()

prompt = (
    "Reformat this table to be a simpler markdown table with "
    + "no hierarchical columns, no pivoting, values and percentages in different columns, "
    + "and no blank cells\n\n"
    + csv_as_str
)

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=0.0,
    prompt=prompt,
    max_tokens=999,
    n=1,
    stop=None,
)

print(completions.choices[0].text)

Markdown(completions.choices[0].text)



| | 0-2 Acres | 2-5 Acres | 5-10 Acres | More than 10 Acres | None | Total |
| --- | --- | --- | --- | --- | --- | --- |
| Chepalungu (Male) | 22 (2.8%) | 6 (.8%) | 1 (.1%) | 0 (0.0%) | 760 (96.3%) | 789 (100.0%) |
| Chepalungu (Female) | 4 (2.2%) | 2 (1.1%) | 0 (0.0%) | 0 (0.0%) | 176 (96.7%) | 182 (100.0%) |
| Bomet Central (Male) | 10 (3.8%) | 2 (.8%) | 0 (0.0%) | 0 (0.0%) | 251 (95.4%) | 263 (100.0%) |
| Bomet Central (Female) | 3 (1.7%) | 0 (0.0%) | 0 (0.0%) | 0 (0.0%) | 170 (98.3%) | 173 (100.0%) |
| Total | 5 (2.9%) | 2 (1.2%) | 1 (.6%) | 0 (0.0%) | 163 (95.3%) | 171 (100.0%) |



| | 0-2 Acres | 2-5 Acres | 5-10 Acres | More than 10 Acres | None | Total |
| --- | --- | --- | --- | --- | --- | --- |
| Chepalungu (Male) | 22 (2.8%) | 6 (.8%) | 1 (.1%) | 0 (0.0%) | 760 (96.3%) | 789 (100.0%) |
| Chepalungu (Female) | 4 (2.2%) | 2 (1.1%) | 0 (0.0%) | 0 (0.0%) | 176 (96.7%) | 182 (100.0%) |
| Bomet Central (Male) | 10 (3.8%) | 2 (.8%) | 0 (0.0%) | 0 (0.0%) | 251 (95.4%) | 263 (100.0%) |
| Bomet Central (Female) | 3 (1.7%) | 0 (0.0%) | 0 (0.0%) | 0 (0.0%) | 170 (98.3%) | 173 (100.0%) |
| Total | 5 (2.9%) | 2 (1.2%) | 1 (.6%) | 0 (0.0%) | 163 (95.3%) | 171 (100.0%) |

It discarded unnecessary rows and converted the data to a nice regular table with column headings, but look closely and you'll see it's lost some key information, such as the breakdown by Male/Female. Classic hallucination territory, it looks so very plausible, but is wrong.

Let's play with the temperature parameter. Lower values make the model more deterministic (giving the same results every time for the same prompt) whereas higher values are more random. With a higher temperature value we get …

In [143]:
prompt = (
    "Reformat this table to be a simpler markdown table with "
    + "no hierarchical columns, no pivoting, values and percentages in different columns, "
    + "and no blank cells\n\n"
    + csv_as_str
)

print(prompt)

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=1.0,
    prompt=prompt,
    max_tokens=999,
    n=1,
    stop=None,
)

Markdown(completions.choices[0].text)


Reformat this table to be a simpler markdown table with no hierarchical columns, no pivoting, values and percentages in different columns, and no blank cells

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,,Sub county,,,,,,,
2,,,,,Chepalungu,,,,Bomet Central,,,
3,,,,,Male,,Female,,Male,,Female,
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
10,,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%




Table 3: Number of acreage under irrigation

|           | 0 - 2 acres | 2 - 5 acres | 5 - 10 acres | More than 10 acres | None | Total |
|-----------|-------------|-------------|--------------|-------------------|------|-------|
| Chepalungu (Male) | 22 (2.8%) | 6 (.8%) | 1 (.1%) | 0 (0.0%) | 760 (96.3%) | 789 (100.0%) |
| Chepalungu (Female) | 4 (2.2%) | 2 (1.1%) | 0 (0.0%) | 0 (0.0%) | 176 (96.7%) | 182 (100.0%) |
| Bomet Central (Male) | 10 (3.8%) | 2 (.8%) | 0 (0.0%) | 0 (0.0%) | 251 (95.4%) | 263 (100.0%) |
| Bomet Central (Female) | 3 (1.7%) | 0 (0.0%) | 0 (0.0%) | 0 (0.0%) | 170 (98.3%) | 173 (100.0%) |
| Overall (Male) | 42 (2.6%) | 10 (1.0%) | 1 (.6%) | 0 (0.0%) | 1261 (95.9%) | 1314 (100.0%) |
| Overall (Female) | 11 (1.5%) | 2 (.9%) | 0 (0.0%) | 0 (0.0%) | 346 (98.1%) | 357 (100.0%) |

Interesting, but still incorrect. Since the temperature is high, let's call again with exactly the same prompt ...

In [39]:
completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=1.0,
    prompt=prompt,
    max_tokens=999,
    n=1,
    stop=None,
)

Markdown(completions.choices[0].text)



Table 3: Number of Acreage Under Irrigation

|  | Average Size of Land | 0 - 2 acres  | 2 - 5 acres  | 5 - 10 acres | More than 10 acres | None    | Total  | 
|---|---------------------|--------------|--------------|---------------|--------------------|--------|--------|
| N | Chepalungu          | 22           | 6            | 1             | 0                  | 760    | 789    |
| % | Chepalungu          | 2.8%         | 0.8%         | 0.1%          | 0.0%               | 96.3%  | 100.0% |
| N | Bomet Central       | 4            | 2            | 0             | 0                  | 176    | 182    |
| % | Bomet Central       | 2.2%         | 1.1%         | 0.0%          | 0.0%               | 96.7%  | 100.0% |
| N | Male                | 10           | 2            | 0             | 0                  | 251    | 263    |
| % | Male                | 3.8%         | 0.8%         | 0.0%          | 0.0%               | 95.4%  | 100.0% |
| N | Female              | 3            | 0            | 0             | 0                  | 170    | 173    |
| % | Female              | 1.7%         | 0.0%         | 0.0%          | 0.0%               | 98.3%  | 100.0% |
| N | Total               | 5            | 2            | 1             | 0                  | 163    | 171    |
| % | Total               | 2.9%         | 1.2%         | 0.6%          | 0.0%               | 95.3%  | 100.0% |

As expected, different results for the same prompt due to the high temperature. The results are non-deterministic.

Interestingly the above captured the column hierarchy quite well, but not rows. 

What about if we pad merged cells?

In [9]:
wb = openpyxl.load_workbook(filename)
sheet = wb["Sheet1"]
merged_table, sheet, hasmerged = pad_merged_cells(sheet)
merged_table.to_csv("test_merged.csv")

display(merged_table)
print(merged_table.to_csv())


,0,1,2,3,4,5,6,7,8,9,10,11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central,Bomet Central
3,,,OVERALL,OVERALL,Male,Male,Female,Female,Male,Male,Female,Female
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that ...,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,What is the average size of land you own that ...,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,What is the average size of land you own that ...,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,What is the average size of land you own that ...,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,What is the average size of land you own that ...,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%


,0,1,2,3,4,5,6,7,8,9,10,11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central,Bomet Central
3,,,OVERALL,OVERALL,Male,Male,Female,Female,Male,Male,Female,Female
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,What is the average size of land you own that is currently under irrigation?,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,What is the average size of land you own that is currently under irrigation?,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,What is the average size of land you own that is currently under irrigation?,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,What is the average size of land you own that is currently under irrigation?,None,76

In [57]:
with open("test_merged.csv", "r") as file:
    csv_as_str_merged = file.read()

prompt = (
    "Reformat this table to be a simpler markdown table with "
    + "no hierarchical columns, no pivoting, values and percentages in different columns, "
    + "and no blank cells\n\n"
    + csv_as_str_merged
)

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=0.0,
    prompt=prompt,
    max_tokens=999,
    n=1,
    stop=None,
)

Markdown(completions.choices[0].text)


|Question|0 - 2 acres (N)|0 - 2 acres (%)|2 - 5 acres (N)|2 - 5 acres (%)|5 - 10 acres (N)|5 - 10 acres (%)|More than 10 acres (N)|More than 10 acres (%)|None (N)|None (%)|Total (N)|Total (%)|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|What is the average size of land you own that is currently under irrigation?|22|2.8%|6|0.8%|1|0.1%|0|0.0%|760|96.3%|789|100.0%|
|Chepalungu (Male)|4|2.2%|2|1.1%|0|0.0%|0|0.0%|176|96.7%|182|100.0%|
|Chepalungu (Female)|10|3.8%|0|0.0%|0|0.0%|0|0.0%|251|95.4%|263|100.0%|
|Bomet Central (Male)|5|2.9%|2|1.2%|1|0.6%|0|0.0%|163|95.3%|171|100.0%|
|Bomet Central (Female)|3|1.7%|0|0.0%|0|0.0%|0|0.0%|170|98.3%|173|100.0%|



|Question|0 - 2 acres (N)|0 - 2 acres (%)|2 - 5 acres (N)|2 - 5 acres (%)|5 - 10 acres (N)|5 - 10 acres (%)|More than 10 acres (N)|More than 10 acres (%)|None (N)|None (%)|Total (N)|Total (%)|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|What is the average size of land you own that is currently under irrigation?|22|2.8%|6|0.8%|1|0.1%|0|0.0%|760|96.3%|789|100.0%|
|Chepalungu (Male)|4|2.2%|2|1.1%|0|0.0%|0|0.0%|176|96.7%|182|100.0%|
|Chepalungu (Female)|10|3.8%|0|0.0%|0|0.0%|0|0.0%|251|95.4%|263|100.0%|
|Bomet Central (Male)|5|2.9%|2|1.2%|1|0.6%|0|0.0%|163|95.3%|171|100.0%|
|Bomet Central (Female)|3|1.7%|0|0.0%|0|0.0%|0|0.0%|170|98.3%|173|100.0%|

In [59]:
completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=1.0,
    prompt=prompt,
    max_tokens=999,
    n=1,
    stop=None,
)

Markdown(completions.choices[0].text)



Table 3: Number of acreage under irrigation

|  |  0 - 2 acres | 2 - 5 acres | 5 - 10 acres | More than 10 acres | None | Total |
|----|--------------|-------------|--------------|--------------------|------|-------|
|OVERALL | 22 (2.8%) | 6 (0.8%) | 1 (0.1%) | 0 (0.0%) | 760 (96.3%) | 789 (100.0%) |
| Chepalungu | 4 (2.2%) | 2 (1.1%) | 0 (0.0%) | 0 (0.0%) | 176 (96.7%) | 182 (100.0%) |
| Bomet Central | 10 (3.8%) | 0 (0%) | 0 (0%) | 0 (0%) | 251 (95.4%) | 263 (100.0%) |
| Male | 5 (2.9%) | 2 (1.2%) |1 (0.6%) | 0 (0.0%) | 163 (95.3%) | 171 (100.0%) |
| Female | 3 (1.7%) | 0 (0.0%) | 0 (0.0%) | 0 (0.0%) | 170 (98.3%) | 173 (100.0%) |

OK, let's perhaps try single-shot, providing an example in the prompt ...

In [14]:
# Example table
wb = openpyxl.load_workbook(prompt_sample_table1, data_only=True)
sheet = wb["Sheet1"]
example_before, sheet, hasmerged = pad_merged_cells(sheet)
example_before_csv = example_before.to_csv()
example_after, hasmerged, report = parse_excel_sheet(sheet)
example_after_markdown = example_after.to_markdown()
example_after_csv = example_after.to_csv()

example_before_csv = """
 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

example_after_markdown = (
    """
 |    |                                      |                                         |   OVERALL - N | OVERALL - %   |   Sub county - Chepalungu | Sub county - Chepalungu - %   |   Sub county - Bomet Central | Sub county - Bomet Central - %   |
|---:|:-------------------------------------|:----------------------------------------|--------------:|:--------------|--------------------------:|:------------------------------|-----------------------------:|:---------------------------------|
|  1 | Infants         on Dietary Diversity | Infants  on  Minimum  Dietary Diversity |            37 | 17.5%         |                        24 | 17.9%                         |                           13 | 16.7%                            |
|  2 | Infants         on Dietary Diversity | Infants not on Dietary Diversity        |           175 | 82.5%         |                       110 | 82.1%                         |                           65 | 83.3%                            |
|  3 | Infants         on Dietary Diversity | Total                                   |           212 | 100.0%        |                       134 | 100.0%                        |                           78 | 100.0%                           |
""".replace(
        ":|", "|"
    )
    .replace("|:", "|")
    .replace("\n", "\n<RETURN>")
)

example_after_csv = """
 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu,Sub county - Chepalungu - %,Sub county - Bomet Central,Sub county - Bomet Central - %
1,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

# filename = './data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx'
# wb = openpyxl.load_workbook(filename)
# sheet = wb['Sheet1']
# table_to_parse_padded, sheet, hasmerged  = pad_merged_cells(sheet)
# print(table_to_parse_padded.to_csv())
# display(table_to_parse_padded)
# sys.exit()

table_to_parse_padded = """
,0,1,2,3,4,5,6,7,8,9,10,11
0,Table 3: Number of acreage under irrigation,,,,,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central,Bomet Central
3,,,OVERALL,OVERALL,Male,Male,Female,Female,Male,Male,Female,Female
4,,,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,What is the average size of land you own that is currently under irrigation?,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,What is the average size of land you own that is currently under irrigation?,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,What is the average size of land you own that is currently under irrigation?,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,What is the average size of land you own that is currently under irrigation?,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
10,What is the average size of land you own that is currently under irrigation?,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%
"""

prompt = (
    "Reformat this table to only have a single header row: \n\n"
    + example_before_csv
    + "\n\n"
    + "Result: \n\n"
    + example_after_csv
    + "\n\n"
    + "Reformat this table to only have a single header row: \n\n"
    + table_to_parse_padded
    + "\n\n"
    + "Result: \n\n"
)

print("\n\n", prompt, "\n\n")

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=0.0,
    prompt=prompt,
    n=1,
    stop=None,
    max_tokens=2068,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

#print("\n========== Actual:\n")
#filename = "./data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx"
#wb = openpyxl.load_workbook(filename)
#sheet = wb["Sheet1"]
#merged_table, hasmerged, report = parse_excel_sheet(sheet, debug=False)
#display(merged_table)
# print("\n========== Input table:\n")
# table_to_parse, sheet, hasmerged = pad_merged_cells(sheet)
print("\n========== Model prediction:\n")

display(pd.read_csv(StringIO(completions.choices[0].text)))


,0,1,2,3,4,5,6,7,8
0,None,None,None,None,None,None,None,None,None
1,Table 16: % of infants on Minimum Dietary Dive...,None,None,None,None,None,None,None,None
2,None,None,None,None,Sub county,Sub county,Sub county,Sub county,Sub county
3,None,None,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central
4,None,None,N,%,n,%,n,%,%
5,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%,16.7%
6,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%,83.3%
7,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%,100.0%



Nulls in first 10 rows: [9, 8, 4, 2, 2, 0, 0, 0]
Numeric first 10 rows: [0, 0, 0, 0, 0, 3, 3, 3]
Unique values in first 10 rows: [0, 1, 1, 3, 3, 8, 8, 6]
Year values in first 10 rows: [0, 0, 0, 0, 0, 0, 0, 0]
HXL row: None

First reduced nulls row: 5
First increased numeric row (excluding years): 5



,,,,,,,,
1,Table 16: % of infants on Minimum Dietary Dive...,,,,,,,
2,,,,,Sub county,Sub county,Sub county,Sub county
3,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central




 Reformat this table to only have a single header row: 


 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%


Result: 


 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu,Sub county - Chepalungu - %,Sub county - Bomet Central,Sub county - Bomet Central - %
1,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,

,,.1,.2,OVERALL - N,OVERALL - %,Sub county - Chepalungu - Male,Sub county - Chepalungu - Male - %,Sub county - Chepalungu - Female,Sub county - Chepalungu - Female - %,Sub county - Bomet Central - Male,Sub county - Bomet Central - Male - %,Sub county - Bomet Central - Female,Sub county - Bomet Central - Female - %
0,1,What is the average size of land you own that ...,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
1,2,What is the average size of land you own that ...,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
2,3,What is the average size of land you own that ...,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
3,4,What is the average size of land you own that ...,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
4,5,What is the average size of land you own that ...,None,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
5,6,What is the average size of land you own that ...,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%


OK, let's prompt with tables unlike the example we provide in the prompt ...

In [88]:
# Example table
wb = openpyxl.load_workbook(prompt_sample_table1, data_only=True)
sheet = wb["Sheet1"]
example_before, sheet, hasmerged = pad_merged_cells(sheet)
# display(example_before)
example_before_csv = example_before.to_csv()
# print("BEFORE:\n", example_before_csv)
example_after, hasmerged, report = parse_excel_sheet(sheet)
example_after_markdown = example_after.to_markdown()
# print("AFTER:\n", example_after_markdown)
example_after_csv = example_after.to_csv()
# print("AFTER:\n", example_after_csv)
# Markdown(example_after_markdown)

example_before_csv = """
 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

example_after_markdown = (
    """
 |    |                                      |                                         |   OVERALL - N | OVERALL - %   |   Sub county - Chepalungu | Sub county - Chepalungu - %   |   Sub county - Bomet Central | Sub county - Bomet Central - %   |
|---:|:-------------------------------------|:----------------------------------------|--------------:|:--------------|--------------------------:|:------------------------------|-----------------------------:|:---------------------------------|
|  1 | Infants         on Dietary Diversity | Infants  on  Minimum  Dietary Diversity |            37 | 17.5%         |                        24 | 17.9%                         |                           13 | 16.7%                            |
|  2 | Infants         on Dietary Diversity | Infants not on Dietary Diversity        |           175 | 82.5%         |                       110 | 82.1%                         |                           65 | 83.3%                            |
|  3 | Infants         on Dietary Diversity | Total                                   |           212 | 100.0%        |                       134 | 100.0%                        |                           78 | 100.0%                           |
""".replace(
        ":|", "|"
    )
    .replace("|:", "|")
    .replace("\n", "\n<RETURN>")
)

example_after_csv = """
 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu - N,Sub county - Chepalungu - %,Sub county - Bomet Central - N,Sub county - Bomet Central - %
1,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

table_to_parse_padded = """
,0,1,2,3,4
0,Random text here,,,,
1,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats
2,TOT,Farm,Small ,Medium,Large
3,97,6666,0.65,,0.13
4,93,Bar M,,0.2,
5,140,Daisy,0.3,0.89,
6,,Plew,0.2,0.5,0.1
7,119,Birdie,0.2,0.1,
8,29,Kit Kat,,0.55,
"""

wb = openpyxl.load_workbook("./data/test_prompt.xlsx", data_only=True)
sheet = wb["Sheet1"]
table_to_parse_padded, sheet, hasmerged = pad_merged_cells(sheet)
print(table_to_parse_padded.to_csv())
table_to_parse_padded = table_to_parse_padded.to_csv()
# sys.exit()

dd = pd.read_csv(StringIO(table_to_parse_padded))
display(dd.fillna(""))

prompt = (
    "Reformat this table to only have a single header row: \n\n"
    + example_before_csv
    + "\n\n"
    + "Result: \n\n"
    + example_after_csv
    + "\n\n"
    + "Reformat this table to only have a single header row: \n\n"
    + table_to_parse_padded
    + "\n\n"
    + "Result: \n\n"
)

print("\n\n", prompt, "\n\n")

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=0.0,
    prompt=prompt,
    n=1,
    stop=None,
    max_tokens=2068,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print("\n========== Actual:\n")

# filename = './data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx'
# wb = openpyxl.load_workbook(filename)
# sheet = wb['Sheet1']
# merged_table, hasmerged, report = parse_excel_sheet(sheet, debug=False)
# display(merged_table)


print("\n========== Input table:\n")
table_to_parse, sheet, hasmerged = pad_merged_cells(sheet)
display(table_to_parse)

print("\n========== Model prediction:\n")

print(completions.choices[0].text)
dd = pd.read_csv(StringIO(completions.choices[0].text))
display(dd.fillna(""))


Sheet1


,0,1,2,3,4,5,6,7,8
0,None,None,None,None,None,None,None,None,None
1,Table 16: % of infants on Minimum Dietary Dive...,None,None,None,None,None,None,None,None
2,None,None,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county,Sub county
3,None,None,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central
4,None,None,N,%,n,%,n,%,%
5,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%,16.7%
6,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%,83.3%
7,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%,100.0%



Nulls in first 10 rows: [9, 8, 2, 2, 2, 0, 0, 0]
Numeric first 10 rows: [0, 0, 0, 0, 0, 3, 3, 3]
Unique values in first 10 rows: [0, 1, 2, 3, 3, 8, 8, 6]
Year values in first 10 rows: [0, 0, 0, 0, 0, 0, 0, 0]
HXL row: None

First reduced nulls row: 5
First increased numeric row (excluding years): 5

Data starts at row: 5



,,,OVERALL - N,OVERALL - %,Sub county - Chepalungu,Sub county - Chepalungu - %,Sub county - Bomet Central,Sub county - Bomet Central - %
1,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%


,0,1,2,3,4,5
0,Random text here,,,,,
1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats
2,,TOT,Farm,Small ,Medium,Large
3,,97,6666,0.65,,0.13
4,,93,Bar M,,0.2,
5,,140,Daisy,0.3,0.89,
6,,,Plew,0.2,0.5,0.1
7,,119,Birdie,0.2,0.1,
8,,29,Kit Kat,,0.55,



,Unnamed: 0,0,1,2,3,4,5
0,0,Random text here,,,,,
1,1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats
2,2,,TOT,Farm,Small,Medium,Large
3,3,,97,6666,0.65,,0.13
4,4,,93,Bar M,,0.2,
5,5,,140,Daisy,0.3,0.89,
6,6,,,Plew,0.2,0.5,0.1
7,7,,119,Birdie,0.2,0.1,
8,8,,29,Kit Kat,,0.55,




 Reformat this table to only have a single header row: 


 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%


Result: 


 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu - N,Sub county - Chepalungu - %,Sub county - Bomet Central - N,Sub county - Bomet Central - %
1,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants         on Dietary Diversity,Total,212,100.0%,134

,0,1,2,3,4,5
0,Random text here,,,,,
1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats
2,,TOT,Farm,Small,Medium,Large
3,,97,6666,0.65,,0.13
4,,93,Bar M,,0.2,
5,,140,Daisy,0.3,0.89,
6,,,Plew,0.2,0.5,0.1
7,,119,Birdie,0.2,0.1,
8,,29,Kit Kat,,0.55,



========== Model prediction:

 , ,Cows - TOT,Cows - Farm,Cowboy Hats - Small,Cowboy Hats - Medium,Cowboy Hats - Large
1,Cows,97,6666,0.65,,0.13
2,Bar M,93,,,0.2,
3,Daisy,140,0.3,0.89,
4,Plew,,0.2,0.5,0.1
5,Birdie,119,0.2,0.1,
6,Kit Kat,29,,0.55,


,,.1,Cows - TOT,Cows - Farm,Cowboy Hats - Small,Cowboy Hats - Medium,Cowboy Hats - Large
0,1,Cows,97.0,6666.0,0.65,,0.13
1,2,Bar M,93.0,,,0.2,
2,3,Daisy,140.0,0.3,0.89,,
3,4,Plew,,0.2,0.5,0.1,
4,5,Birdie,119.0,0.2,0.1,,
5,6,Kit Kat,29.0,,0.55,,


In [109]:
# Example table
wb = openpyxl.load_workbook(prompt_sample_table1, data_only=True)
sheet = wb["Sheet1"]
example_before = pad_merged_cells(sheet)
example_before_csv = example_before.to_csv()
example_after, hasmerged, report = parse_excel_sheet(sheet)
example_after_markdown = example_after.to_markdown()
example_after_csv = example_after.to_csv()

example_before_csv = """
 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

dd = pd.read_csv(StringIO(example_before_csv))
dd = dd.T.fillna("")
example_before_cols = dd.to_csv()

example_after_markdown = (
    """
 |    |                                      |                                         |   OVERALL - N | OVERALL - %   |   Sub county - Chepalungu | Sub county - Chepalungu - %   |   Sub county - Bomet Central | Sub county - Bomet Central - %   |
|---:|:-------------------------------------|:----------------------------------------|--------------:|:--------------|--------------------------:|:------------------------------|-----------------------------:|:---------------------------------|
|  1 | Infants         on Dietary Diversity | Infants  on  Minimum  Dietary Diversity |            37 | 17.5%         |                        24 | 17.9%                         |                           13 | 16.7%                            |
|  2 | Infants         on Dietary Diversity | Infants not on Dietary Diversity        |           175 | 82.5%         |                       110 | 82.1%                         |                           65 | 83.3%                            |
|  3 | Infants         on Dietary Diversity | Total                                   |           212 | 100.0%        |                       134 | 100.0%                        |                           78 | 100.0%                           |
""".replace(
        ":|", "|"
    )
    .replace("|:", "|")
    .replace("\n", "\n<RETURN>")
)

example_after_csv = """
 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu - N,Sub county - Chepalungu - %,Sub county - Bomet Central - N,Sub county - Bomet Central - %
1,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

# wb = openpyxl.load_workbook('test_prompt.xlsx', data_only=True)
# sheet = wb['Sheet1']
# table_to_parse_padded = pad_merged_cells(sheet)
# print(table_to_parse_padded.to_csv())
# table_to_parse_padded = table_to_parse_padded.to_csv()
# sys.exit()

table_to_parse_padded = """
,0,1,2,3,4,5
0,Random text here,,,,,
1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats
2,,TOT,Farm,Small ,Medium,Large
3,,97,6666,0.65,,0.13
4,,93,Bar M,,0.2,
5,,140,Daisy,0.3,0.89,
6,,,Plew,0.2,0.5,0.1
7,,119,Birdie,0.2,0.1,
8,,29,Kit Kat,,0.55,
"""

dd = pd.read_csv(StringIO(table_to_parse_padded)).fillna("")
display(dd)

# dd = dd.T.fillna('')
# table_to_be_parsed_cols = dd.to_csv()

prompt = (
    "We need to reformat this table to only have a single header row: \n\n"
    + example_before_csv
    + "\n"
    + "Let's think step by step \n"
    + "Row 1 is just an index row, it has no text or data \n"
    + "Row 2 contains just label text \n"
    + "Rows 3 to 5 contain column headers \n"
    + "Rows 6 onwards contain data \n"
    + "Columns are separated by commas, there should be 7 commas on each row \n"
    + "If we combine each colummn of rows 3 to 5 by concatenating vertically, we get \n"
    + example_after_csv
    + "\n\n"
    + "We need to reformat this table to only have a single header row: \n\n"
    + table_to_parse_padded
    + "\n\n"
    + "Let's think step by step \n\n"
)

print("\n\n", prompt, "\n\n")

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=0.0,
    prompt=prompt,
    n=1,
    stop=None,
    max_tokens=2068,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print("\n========== Actual:\n")

# filename = './data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx'
# wb = openpyxl.load_workbook(filename)
# sheet = wb['Sheet1']
# merged_table, hasmerged, report = parse_excel_sheet(sheet, debug=False)
# display(merged_table)


# print("\n========== Input table:\n")
# table_to_parse = pad_merged_cells(sheet)
# display(table_to_parse)

print("\n========== Model prediction:\n")

print(completions.choices[0].text)
# display(pd.read_csv(StringIO(completions.choices[0].text)))


Sheet1


,0,1,2,3,4,5,6,7,8
0,None,None,None,None,None,None,None,None,None
1,Table 16: % of infants on Minimum Dietary Dive...,None,None,None,None,None,None,None,None
2,None,None,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county,Sub county
3,None,None,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central
4,None,None,N,%,n,%,n,%,%
5,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%,16.7%
6,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%,83.3%
7,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%,100.0%



Nulls in first 10 rows: [9, 8, 2, 2, 2, 0, 0, 0]
Numeric first 10 rows: [0, 0, 0, 0, 0, 3, 3, 3]
Unique values in first 10 rows: [0, 1, 2, 3, 3, 8, 8, 6]
Year values in first 10 rows: [0, 0, 0, 0, 0, 0, 0, 0]
HXL row: None

First reduced nulls row: 5
First increased numeric row (excluding years): 5

Data starts at row: 5



,,,OVERALL - N,OVERALL - %,Sub county - Chepalungu,Sub county - Chepalungu - %,Sub county - Bomet Central,Sub county - Bomet Central - %
1,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%


,Unnamed: 0,0,1,2,3,4,5
0,0,Random text here,,,,,
1,1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats
2,2,,TOT,Farm,Small,Medium,Large
3,3,,97,6666,0.65,,0.13
4,4,,93,Bar M,,0.2,
5,5,,140,Daisy,0.3,0.89,
6,6,,,Plew,0.2,0.5,0.1
7,7,,119,Birdie,0.2,0.1,
8,8,,29,Kit Kat,,0.55,




 We need to reformat this table to only have a single header row: 


 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%

Let's think step by step 
Row 1 is just an index row, it has no text or data 
Row 2 contains just label text 
Rows 3 to 5 contain column headers 
Rows 6 onwards contain data 
Columns are separated by commas, there should be 7 commas on each row 
If we combine each colummn of rows 3 to 5 by concatenating vertically, we get 

 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu - N,Sub county - Chepalungu - %,Sub

In [111]:
result = """
 , ,Cows - TOT,Cows - Farm,Cowboy Hats - Small ,Cowboy Hats - Medium,Cowboy Hats - Large
1,,97,6666,0.65,,0.13
2,,93,Bar M,,0.2,
3,,140,Daisy,0.3,0.89,
4,,,Plew,0.2,0.5,0.1
5,,119,Birdie,0.2,0.1,
6,,29,Kit Kat,,0.55,
"""
dd = pd.read_csv(StringIO(result)).fillna("")
display(dd)


,,.1,Cows - TOT,Cows - Farm,Cowboy Hats - Small,Cowboy Hats - Medium,Cowboy Hats - Large
0,1,,97.0,6666,0.65,,0.13
1,2,,93.0,Bar M,,0.2,
2,3,,140.0,Daisy,0.3,0.89,
3,4,,,Plew,0.2,0.5,0.1
4,5,,119.0,Birdie,0.2,0.1,
5,6,,29.0,Kit Kat,,0.55,


Add an extra column ...

In [115]:
# Example table
wb = openpyxl.load_workbook(prompt_sample_table1, data_only=True)
sheet = wb["Sheet1"]
example_before = pad_merged_cells(sheet)
# display(example_before)
example_before_csv = example_before.to_csv()
# print("BEFORE:\n", example_before_csv)
example_after, hasmerged, report = parse_excel_sheet(sheet)
example_after_markdown = example_after.to_markdown()
# print("AFTER:\n", example_after_markdown)
example_after_csv = example_after.to_csv()
# print("AFTER:\n", example_after_csv)
# Markdown(example_after_markdown)

example_before_csv = """
 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

dd = pd.read_csv(StringIO(example_before_csv))
dd = dd.T.fillna("")
example_before_cols = dd.to_csv()

example_after_markdown = (
    """
 |    |                                      |                                         |   OVERALL - N | OVERALL - %   |   Sub county - Chepalungu | Sub county - Chepalungu - %   |   Sub county - Bomet Central | Sub county - Bomet Central - %   |
|---:|:-------------------------------------|:----------------------------------------|--------------:|:--------------|--------------------------:|:------------------------------|-----------------------------:|:---------------------------------|
|  1 | Infants         on Dietary Diversity | Infants  on  Minimum  Dietary Diversity |            37 | 17.5%         |                        24 | 17.9%                         |                           13 | 16.7%                            |
|  2 | Infants         on Dietary Diversity | Infants not on Dietary Diversity        |           175 | 82.5%         |                       110 | 82.1%                         |                           65 | 83.3%                            |
|  3 | Infants         on Dietary Diversity | Total                                   |           212 | 100.0%        |                       134 | 100.0%                        |                           78 | 100.0%                           |
""".replace(
        ":|", "|"
    )
    .replace("|:", "|")
    .replace("\n", "\n<RETURN>")
)

example_after_csv = """
 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu - N,Sub county - Chepalungu - %,Sub county - Bomet Central - N,Sub county - Bomet Central - %
1,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%
"""

# wb = openpyxl.load_workbook('test_prompt.xlsx', data_only=True)
# sheet = wb['Sheet1']
# table_to_parse_padded = pad_merged_cells(sheet)
# print(table_to_parse_padded.to_csv())
# table_to_parse_padded = table_to_parse_padded.to_csv()
# sys.exit()

table_to_parse_padded = """
,0,1,2,3,4,5,6
0,Random text here,,,,,,
1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats, Organic
2,,TOT,Farm,Small ,Medium,Large, Organic
3,,97,6666,0.65,,0.13,Y
4,,93,Bar M,,0.2,,Y
5,,140,Daisy,0.3,0.89,,N
6,,,Plew,0.2,0.5,0.1,Y
7,,119,Birdie,0.2,0.1,,N
8,,29,Kit Kat,,0.55,,Y
"""

dd = pd.read_csv(StringIO(table_to_parse_padded)).fillna("")
display(dd)

# dd = dd.T.fillna('')
# table_to_be_parsed_cols = dd.to_csv()

prompt = (
    "We need to reformat this table to only have a single header row: \n\n"
    + example_before_csv
    + "\n"
    + "Let's think step by step \n"
    + "Row 1 is just an index row, it has no text or data \n"
    + "Row 2 contains just label text \n"
    + "Rows 3 to 5 contain column headers \n"
    + "Rows 6 onwards contain data \n"
    + "Columns are separated by commas, there should be 7 commas on each row \n"
    + "If we combine each colummn of rows 3 to 5 by concatenating vertically, we get \n"
    + example_after_csv
    + "\n\n"
    + "We need to reformat this table to only have a single header row: \n\n"
    + table_to_parse_padded
    + "\n\n"
    + "Let's think step by step \n\n"
)

print("\n\n", prompt, "\n\n")

completions = ai.Completion.create(
    engine="text-davinci-003",
    temperature=0.0,
    prompt=prompt,
    n=1,
    stop=None,
    max_tokens=2068,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# print("\n========== Actual:\n")
# filename = './data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx'
# wb = openpyxl.load_workbook(filename)
# sheet = wb['Sheet1']
# merged_table, hasmerged, report = parse_excel_sheet(sheet, debug=False)
# display(merged_table)
# print("\n========== Input table:\n")
# table_to_parse = pad_merged_cells(sheet)
# display(table_to_parse)
# print("\n========== Model prediction:\n")

print(completions.choices[0].text)


Sheet1


,0,1,2,3,4,5,6,7,8
0,None,None,None,None,None,None,None,None,None
1,Table 16: % of infants on Minimum Dietary Dive...,None,None,None,None,None,None,None,None
2,None,None,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county,Sub county
3,None,None,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central,Bomet Central
4,None,None,N,%,n,%,n,%,%
5,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%,16.7%
6,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%,83.3%
7,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%,100.0%



Nulls in first 10 rows: [9, 8, 2, 2, 2, 0, 0, 0]
Numeric first 10 rows: [0, 0, 0, 0, 0, 3, 3, 3]
Unique values in first 10 rows: [0, 1, 2, 3, 3, 8, 8, 6]
Year values in first 10 rows: [0, 0, 0, 0, 0, 0, 0, 0]
HXL row: None

First reduced nulls row: 5
First increased numeric row (excluding years): 5

Data starts at row: 5



,,,OVERALL - N,OVERALL - %,Sub county - Chepalungu,Sub county - Chepalungu - %,Sub county - Bomet Central,Sub county - Bomet Central - %
1,Infants on Dietary Diversity,Infants on Minimum Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
2,Infants on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
3,Infants on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%


,Unnamed: 0,0,1,2,3,4,5,6
0,0,Random text here,,,,,,
1,1,,Cows,Cows,Cowboy Hats,Cowboy Hats,Cowboy Hats,Organic
2,2,,TOT,Farm,Small,Medium,Large,Organic
3,3,,97,6666,0.65,,0.13,Y
4,4,,93,Bar M,,0.2,,Y
5,5,,140,Daisy,0.3,0.89,,N
6,6,,,Plew,0.2,0.5,0.1,Y
7,7,,119,Birdie,0.2,0.1,,N
8,8,,29,Kit Kat,,0.55,,Y




 We need to reformat this table to only have a single header row: 


 ,0,1,2,3,4,5,6,7
0,Table 16: % of infants on Minimum Dietary Diversity,,,,,,,
1,,,OVERALL,OVERALL,Sub county,Sub county,Sub county,Sub county
2,,,OVERALL,OVERALL,Chepalungu,Chepalungu,Bomet Central,Bomet Central
3,,,N,%,n,%,n,%
4,Infants         on Dietary Diversity,Infants  on  Minimum  Dietary Diversity,37,17.5%,24,17.9%,13,16.7%
5,Infants         on Dietary Diversity,Infants not on Dietary Diversity,175,82.5%,110,82.1%,65,83.3%
6,Infants         on Dietary Diversity,Total,212,100.0%,134,100.0%,78,100.0%

Let's think step by step 
Row 1 is just an index row, it has no text or data 
Row 2 contains just label text 
Rows 3 to 5 contain column headers 
Rows 6 onwards contain data 
Columns are separated by commas, there should be 7 commas on each row 
If we combine each colummn of rows 3 to 5 by concatenating vertically, we get 

 , , ,OVERALL - N,OVERALL - %,Sub county - Chepalungu - N,Sub county - Chepalungu - %,Sub

In [116]:
result = """
, , ,Cows - TOT,Cows - Farm,Cowboy Hats - Small ,Cowboy Hats - Medium,Cowboy Hats - Large, Organic
1,,97,6666,0.65,,0.13,Y
2,,93,Bar M,,0.2,,Y
3,,140,Daisy,0.3,0.89,,N
4,,,Plew,0.2,0.5,0.1,Y
5,,119,Birdie,0.2,0.1,,N
6,,29,Kit Kat,,0.55,,Y
"""
dd = pd.read_csv(StringIO(result)).fillna("")
display(dd)


,Unnamed: 0,,.1,Cows - TOT,Cows - Farm,Cowboy Hats - Small,Cowboy Hats - Medium,Cowboy Hats - Large,Organic
0,1,,97.0,6666,0.65,,0.13,Y,
1,2,,93.0,Bar M,,0.2,,Y,
2,3,,140.0,Daisy,0.3,0.89,,N,
3,4,,,Plew,0.2,0.5,0.1,Y,
4,5,,119.0,Birdie,0.2,0.1,,N,
5,6,,29.0,Kit Kat,,0.55,,Y,


## Fine tuning GPT-3

Here we will generate some prompts for fine-tuning by using some rule-based parsing. The following assumes we have downloaded lots of Excel files from HDX, see [previous blog post](https://medium.com/towards-data-science/predicting-metadata-for-humanitarian-datasets-using-gpt-3-b104be17716d) for details.

In [28]:
resources = pd.read_pickle(hdx_resources_pkl_file)

display(resources.head())
print(resources.shape)
df = resources[(resources["resource_format"] == "XLSX")][
    ["resource_format", "file", "sheet", "dataset_name", "dataset_org_title"]
]
df.drop_duplicates(inplace=True)

orgs = df["dataset_org_title"].unique()

print("Unique orgs: ", len(orgs))


,resource_id,resource_name,resource_format,dataset_id,dataset_name,dataset_org_title,dataset_last_modified,dataset_tags,dataset_groups,dataset_total_res_downloads,...,type,dataset,sheet,resource,num_rows,num_cols,columns,first_row,has_hxl_header,first_nrows
0,be27b14b-217d-4a2b-a172-0ad46ee0b2cc,Transport & Migration Features_Igambang'ombe a...,CSV,d280f085-aa1f-452c-bda2-085d3cde3557,georeferenced-sectorial-data-for-igambang-ombe...,International Federation of Red Cross and Red ...,2023-01-17T08:51:54.588977,"[{'display_name': 'agriculture-livestock', 'id...","[{'description': '', 'display_name': 'Kenya', ...",56,...,csv,"{'archived': False, 'caveats': 'This is primar...",,{'alt_url': 'https://data.humdata.org/dataset/...,56,10,"['Date', 'Subcounty', 'Wards', 'Sublocation', ...","['5/17/2022', ""Igambang'ombe"", 'Mariani', 'Wer...",False,Date Subcounty Wards ...
1,56c55273-3413-4617-966c-0ca0ab3175ad,Agriculture and Livestock features_Igambang'om...,CSV,d280f085-aa1f-452c-bda2-085d3cde3557,georeferenced-sectorial-data-for-igambang-ombe...,International Federation of Red Cross and Red ...,2023-01-17T08:51:54.588977,"[{'display_name': 'agriculture-livestock', 'id...","[{'description': '', 'display_name': 'Kenya', ...",56,...,csv,"{'archived': False, 'caveats': 'This is primar...",,{'alt_url': 'https://data.humdata.org/dataset/...,86,11,"['Date', 'Subcounty', 'Wards', 'Sublocation', ...","['5/21/2022', ""Igambang'ombe"", ""Igambang'ombe""...",False,Date Subcounty Wards Su...
2,c7ef8c53-476b-47bc-9b54-e2b75667066e,Health facilities_Igambang'ombe and Tharaka No...,CSV,d280f085-aa1f-452c-bda2-085d3cde3557,georeferenced-sectorial-data-for-igambang-ombe...,International Federation of Red Cross and Red ...,2023-01-17T08:51:54.588977,"[{'display_name': 'agriculture-livestock', 'id...","[{'description': '', 'display_name': 'Kenya', ...",56,...,csv,"{'archived': False, 'caveats': 'This is primar...",,{'alt_url': 'https://data.humdata.org/dataset/...,52,21,"['Date', 'Subcounty', 'Wards', 'Sublocation', ...","['5/17/2022', ""Igambang'ombe"", 'Mariani', 'Wer...",False,Date Subcounty Wards ...
3,351b6f80-4c6f-4a2d-992f-ff523ad0b7af,Places of Worship_Igambang'ombe and Tharaka No...,CSV,d280f085-aa1f-452c-bda2-085d3cde3557,georeferenced-sectorial-data-for-igambang-ombe...,International Federation of Red Cross and Red ...,2023-01-17T08:51:54.588977,"[{'display_name': 'agriculture-livestock', 'id...","[{'description': '', 'display_name': 'Kenya', ...",56,...,csv,"{'archived': False, 'caveats': 'This is primar...",,{'alt_url': 'https://data.humdata.org/dataset/...,397,12,"['Date', 'Subcounty', 'Wards', 'Sublocation', ...","['5/17/2022', ""Igambang'ombe"", ""Igambang'ombe""...",False,Date Subcounty Wards Su...
4,8400b5bc-bf76-40c0-bb9c-1fc00ede652e,Education Sector_Igambang'ombe and Tharaka Nor...,CSV,d280f085-aa1f-452c-bda2-085d3cde3557,georeferenced-sectorial-data-for-igambang-ombe...,International Federation of Red Cross and Red ...,2023-01-17T08:51:54.588977,"[{'display_name': 'agriculture-livestock', 'id...","[{'description': '', 'display_name': 'Kenya', ...",56,...,csv,"{'archived': False, 'caveats': 'This is primar...",,{'alt_url': 'https://data.humdata.org/dataset/...,169,16,"['Date', 'Subcounty', 'Wards', 'Sublocation', ...","['5/17/2022', ""Igambang'ombe"", ""Igambang'ombe""...",False,Date Subcounty Wards Su...


(1458, 22)
Unique orgs:  30


In [ ]:
prompts = []

# Loop by dataset source so we train on sources different to prediction, otherwise very simil.ar
datasets = (
    df.groupby(by=["dataset_org_title"])["resource_format"]
    .count()
    .reset_index()
    .sort_values("resource_format", ascending=True)
)

maxn = 15

for dataset_org in orgs[0:max_dataset_orgs]:

    print(f"Organization {dataset_org}")
    rows = df.loc[df["dataset_org_title"] == dataset_org]
    for index, row in rows.iterrows():
        filename = row["file"]
        sheetname = row["sheet"]

        if "ESARO Regional db" in filename:
            print(
                "Skipping ESARO Regional db as it's full of multiple tables on sheets ..."
            )
            continue

        print(filename, sheetname)

        wb = openpyxl.load_workbook(filename, data_only=True)
        sheet = wb[sheetname]
        merged_table, hasmerged, report = parse_excel_sheet(
            sheet, maxn, debug=False)

        report_elements = report.split("\n\n")
        prompt = report_elements[0] + "\n" + report_elements[1]
        completion = report_elements[2]

        # Skip a few with known challenges
        if "LIKELY PARSING ERRORS" in str(merged_table.to_csv()):
            print("Skipping multitable ...")
            continue

        p = {
            "prompt": prompt,
            "completion": completion,
            "meta_data": filename,  # We will remove this later, but needed to prune the data
        }
        prompts.append(p)

        # Debug - only output for more compplicated tables for review
        if int(completion.split(":")[1].strip()) > 2:
            print(
                f"============================ {filename} {sheetname} START ============================"
            )
            print(f"\n{report}")
            before = pad_merged_cells(sheet)
            after = merged_table
            display(before.iloc[0:5])
            display(after.iloc[0:3])
            print(
                f"============================ {filename} {sheetname} START ============================"
            )

with open("prompts.json", "w") as f:
    for p in prompts:
        json.dump(p, f)
        f.write("\n")


OK those parsed, let's use them to create a fine-tuning run ...

In [81]:
# https://github.com/openai/openai-cookbook/blob/main/examples/azure/finetuning.ipynb

ai.api_key_path = "./api_key.txt"

train_file = "./prompts.json"

prompts = pd.read_json(path_or_buf=train_file, lines=True)
with open("prompts_input.json", "w") as f:
    for i, row in prompts.iterrows():
        p = {"prompt": row["prompt"], "completion": row["completion"]}
        json.dump(p, f)
        f.write("\n")
train_file = "./prompts_input.json"

print("Uploading training file ...")
training_id = cli.FineTune._get_or_upload(train_file, True)
# validation_id = cli.FineTune._get_or_upload(validation_file_name, True)

print("Fine-tuning model ...")
create_args = {
    "training_file": training_id,
    # "model": "ada"
    "model": "davinci",
}
# https://beta.openai.com/docs/api-reference/fine-tunes/create
resp = ai.FineTune.create(**create_args)
job_id = resp["id"]
status = resp["status"]

print(f"Fine-tunning model with jobID: {job_id}.")


Uploading training file ...


Upload progress: 100%|██████████| 100k/100k [00:00<00:00, 152Mit/s]


Uploaded file from ./prompts_input.json: file-BMSFPbk5piIxHXX2kN5FeZgr
Fine-tuning model ...
Fine-tunning model with jobID: ft-3zWpsoitcKBsu2INkd7wD1qt.


In [96]:
ai.api_key_path = "./api_key.txt"

# ai.FineTune.cancel(id = job_id)

result = ai.FineTune.retrieve(id=job_id)

print(result["status"])
print(result)


succeeded
{
  "created_at": 1677025444,
  "events": [
    {
      "created_at": 1677025444,
      "level": "info",
      "message": "Created fine-tune: ft-3zWpsoitcKBsu2INkd7wD1qt",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1677037247,
      "level": "info",
      "message": "Fine-tune costs $4.79",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1677037248,
      "level": "info",
      "message": "Fine-tune enqueued",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1677042777,
      "level": "info",
      "message": "Fine-tune is in the queue. Queue number: 31",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1677042852,
      "level": "info",
      "message": "Fine-tune is in the queue. Queue number: 30",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1677042926,
      "level": "info",
      "message": "Fine-tune is in the queue. Queue number: 29",
      "object": "fine-tune-event"
 

In [97]:
# Once model is done, this should return model for inference
model = result["fine_tuned_model"]

print(model)


davinci:ft-personal-2023-02-22-06-03-57


In [79]:
# print(ai.FineTune.list())
# ai.api_key_path="./api_key.txt"
# result = openai.FineTune.retrieve(id=job_id)
# print(result)

In [ ]:
# Loop by dataset source so we train on sources different to prediction, otherwise very simil.ar
datasets = (
    df.groupby(by=["dataset_org_title"])["resource_format"]
    .count()
    .reset_index()
    .sort_values("resource_format", ascending=True)
)

results = []
max_results = 200
maxn = 15

for dataset_org in orgs[max_dataset_orgs + 1:]:
    print(f"Organization {dataset_org}")
    rows = df.loc[df["dataset_org_title"] == dataset_org]
    row = rows.iloc[0]  # Take one sheet from each org to get more variation
    print(
        f"\n================================================== {dataset_org} ====================================================\n"
    )
    if len(results) > max_results:
        break
    filename = row["file"]
    sheetname = row["sheet"]

    if "ESARO Regional db" in filename:
        print(
            "Skipping ESARO Regional db as it's full of multiple tables on sheets ..."
        )
        continue

    # Try/Except needed because some xlsx files don't seem to load
    wb = openpyxl.load_workbook(filename, data_only=True)
    for s in wb.sheetnames:
        print(f"Processing sheet {s} ...")
        sheet = wb[s]
        before = pad_merged_cells(sheet)

        merged_table, hasmerged, report = parse_excel_sheet(
            sheet, maxn, debug=False)

        # Skip sheets that have multiple tables on them (this exclusion is mentioned in Blog post)
        if "LIKELY PARSING ERRORS" in str(merged_table.to_csv()):
            print("Skipping multitable ...")
            continue

        report_elements = report.split("\n\n")
        prompt = report_elements[0] + report_elements[1]
        completion = report_elements[2]

        res = make_gpt3_prediction(prompt, model, temperature=0.0)

        predicted = res["predicted"].split(":")[1].strip()
        actual = completion.split(":")[1].strip()

        print(
            f"\n------------------------------ {filename} {sheetname} START ----------------------------------\n"
        )
        print(f"\nPrompt: {prompt}")
        print(f"Actual: {actual}")
        print(f"Predicted: {predicted}\n")
        if actual != predicted:
            display(before.iloc[0:10])
            display(merged_table.iloc[0:10])
        print(
            "\n----------------------------------------- END ------------------------------------------------\n"
        )

        results.append(
            {"prompt": prompt, "predicted": predicted, "expected": actual})


output_prediction_metrics(results, prediction_field="predicted")


## Putting it all together

In [30]:
model = "davinci:ft-personal-2023-02-22-06-03-57"

filename = "./data/Kenya/kenya-mpi_f27114d0-4094-4117-8eb7-19b815358f1c/KEN_xlsx_b2f0fc05_3897_4366_a4a9_c2dd167b5ea5/KEN.xlsx"
wb = openpyxl.load_workbook(filename, data_only=True)
sheet = wb["MPI Region"]
print("BEFORE:")
display(pd.DataFrame(sheet.values))
merged_table, hasmerged, report = parse_excel_sheet(
    sheet, maxn=15, model=model, debug=False
)
print("AFTER:")
display(merged_table)

merged_table.to_excel("test.xlsx")


BEFORE:


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,MPI results by subnational regions,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,This table reports the MPI estimates for subna...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"Citation: Alkire, S., Kanagaratnam, U., and Su...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,ISO\ncountry numeric code,ISO\ncountry code,Country,World region,MPI data source,None,Subnational \nregion,MPI of the country,Multidimensional poverty by region,None,...,None,None,Total population by countryᵃ,None,None,Population 2020,None,None,Indicators included \nin the MPI,None
5,None,None,None,None,None,None,None,None,Multidimensional Poverty Index\n(MPI = H*A),Headcount ratio: Population in multidimensiona...,...,Vulnerable to poverty,In severe poverty,Year of the survey,Population 2019,Population 2020,Population share by region,Population size by \nregion,Number of MPI poor by regionᵇ,Total number of indicators included \n(out of ...,Indicator (s) missing
6,None,None,None,None,Survey,Year,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,Range 0 to 1,Range 0 to 1,% Population,...,% Population,% Population,Thousands,Thousands,Thousands,% Population,Thousands,Thousands,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Central,0.170776,0.073547,18.272351,...,35.416675,2.473355,45831.863,50951.45,51985.78,11.166251,5804.862793,1060.684937,10,


Call GPT-3 to find first data row
Nulls in first 15 rows: [20, 20, 20, 21, 0, 0, 0, 0, 21, 0, 0, 0, 0, 0, 0]
Numeric first 15 rows: [0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 14, 14, 14, 14, 14]
Unique values in first 15 rows: [1, 1, 1, 0, 11, 20, 21, 13, 0, 21, 21, 21, 21, 21, 21]
Year values in first 15 rows: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
HXL row: None

First reduced nulls row: 4
First increased numeric row (excluding years): 9


Data starts at row: 
GPT-3 prediction: 9
AFTER:


,ISO\ncountry numeric code -,ISO\ncountry code -,Country -,World region -,MPI data source - Survey -,MPI data source - Year -,Subnational \nregion -,MPI of the country - Range 0 to 1 -,Multidimensional poverty by region - Multidimensional Poverty Index\n(MPI = H*A) - Range 0 to 1 -,Multidimensional poverty by region - Headcount ratio: Population in multidimensional poverty\n(H) - % Population -,...,Multidimensional poverty by region - Vulnerable to poverty - % Population -,Multidimensional poverty by region - In severe poverty - % Population -,Total population by countryᵃ - Year of the survey - Thousands -,Total population by countryᵃ - Population 2019 - Thousands -,Total population by countryᵃ - Population 2020 - Thousands -,Population 2020 - Population share by region - % Population -,Population 2020 - Population size by \nregion - Thousands -,Population 2020 - Number of MPI poor by regionᵇ - Thousands -,Indicators included \nin the MPI - Total number of indicators included \n(out of ten) -,Indicators included \nin the MPI - Indicator (s) missing -
1,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Central,0.170776,0.073547,18.272351,...,35.416675,2.473355,45831.863,50951.45,51985.78,11.166251,5804.862793,1060.684937,10,
2,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Coast,0.170776,0.212291,44.696212,...,28.525466,18.887437,45831.863,50951.45,51985.78,9.800696,5094.968262,2277.257812,10,
3,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Eastern,0.170776,0.178086,40.134313,...,41.996209,12.782841,45831.863,50951.45,51985.78,14.937041,7765.137207,3116.484619,10,
4,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Nairobi,0.170776,0.028095,7.069267,...,13.331487,1.593338,45831.863,50951.45,51985.78,8.690326,4517.733887,319.370667,10,
5,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,North Eastern,0.170776,0.484089,84.57712,...,9.111195,60.403009,45831.863,50951.45,51985.78,2.852737,1483.017334,1254.293335,10,
6,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Nyanza,0.170776,0.164907,38.811081,...,43.851556,8.966313,45831.863,50951.45,51985.78,14.111913,7336.187988,2847.253906,10,
7,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Rift Valley,0.170776,0.205707,44.189331,...,35.060482,15.77782,45831.863,50951.45,51985.78,26.608951,13832.871094,6112.65332,10,
8,404,KEN,Kenya,Sub-Saharan Africa,DHS,2014,Western,0.170776,0.176617,40.559055,...,49.250714,9.107839,45831.863,50951.45,51985.78,11.832084,6151.001465,2494.788086,10,
9,Notes,,,,,,,,,,...,,,,,,,,,,
10,"ᵃUnited Nations, Department of Economic and So...",,,,,,,,,,...,,,,,,,,,,


In [ ]:
# filename = './data/Kenya/kenya-number-of-acreage-under-irrigation-in-bomet-county_f022a4ea-181b-423d-ab70-53429bad05e2/Number_of_Acreage_under_Irrigation_xlsx_480cf688_857c_47fe_8cb2_712f17ab4619/Number of Acreage under Irrigation.xlsx'
# filename='./data/Kenya/primary-health-care-soins-de-sante-primaire_d86fa64b-d43c-44a2-9486-71e096578575/Primary_health_care_Soins_de_sante_primaire_xlsx_a5f765e0_69ba_4a0f_860b_84c51c008bce/Primary health care  - Soins de sante primaire.xlsx'
# filename ='./data/Kenya/kenya-gross-county-product-gcp-by-economic-activities-per-county_5d7595e8-d66e-49b3-8715-66eedd87a170/GCP_per_County_Current_Prices_2013_2017_xlsx_1631b330_d576_4401_8ba9_94d2a3c6d4b5/GCP per County, Current Prices (2013-2017).xlsx'
# filename='./data/Kenya/sind-protection-in-danger-monthly-news-briefs-dataset_14b1ad1d-abcf-453d-970c-cb964084134e/2020_2022_Protection_in_Danger_Incident_Data__0bee30d9_f045_474a_bd21_ce49140ff0f8/2020-2022 Protection in Danger Incident Data..xlsx'
# filename='./data/Kenya/kenya-monthly-retail-prices-for-dry-maize_d1e200cc-51b0-4857-9ed8-98cc441c9467/Monthly_Retail_Prices_for_Dry_Maize_2017_per_Kg_xlsx_664e259e_4abb_4327_aba7_85f784ff9e3a/Monthly Retail Prices for Dry Maize, 2017 (per Kg).xlsx'
# filename ='./data/Kenya/kenya-special-needs-for-primary-school-children-per-county_c5eda6b1-0d91-4fad-a493-31254b15db2a/Overloaded_Desks_per_county_xlsx_9320755d_133b_4bb1_ab23_cecbc0bc1e2b/Overloaded Desks per county.xlsx'
# filename='./data/Kenya/total-migrants-total-migrants_c8463852-56d5-4e7e-8fba-85bf97370a31/POPULATION_migration_xlsx_582e2c94_d2f0_44d1_873e_3c72f98167af/POPULATION-migration.xlsx'
# filename = './data/Kenya/kenya-special-needs-for-primary-school-children-per-county_c5eda6b1-0d91-4fad-a493-31254b15db2a/Overloaded_Desks_per_county_xlsx_9320755d_133b_4bb1_ab23_cecbc0bc1e2b/Overloaded Desks per county.xlsx'
# filename='./data/Kenya/eastern-and-southern-africa-humanitarian-update-october-2019_be8c0dfb-95a9-44ef-a9f1-d2e2539d4ead/UNICEF_ESARO_Regional_db_2019_Feb_2_7_2020_xlsx_73a575cc_8a0d_4f5f_9734_756de9cd2f87/UNICEF ESARO Regional db 2019 Feb 2.7.2020.xlsx'
filename = "./data/Kenya/kenya-mpi_f27114d0-4094-4117-8eb7-19b815358f1c/KEN_xlsx_b2f0fc05_3897_4366_a4a9_c2dd167b5ea5/KEN.xlsx"
wb = openpyxl.load_workbook(filename, data_only=True)
for s in wb.sheetnames:
    sheet = wb[s]
    merged_table, hasmerged, report = parse_excel_sheet(sheet)
